In [ ]:

import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# 1) Read your Excel and CSV files
df1 = pd.read_excel("name_table.xls")  # Contains column 'botanical name'
df2 = pd.read_csv("bsi_plant_data_formatted.csv")  # Contains column 'Plant Name'

# 2) Load the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# 3) Create embeddings for each row in df1 and df2
df1['embedding'] = df1['botanical name'].apply(lambda x: model.encode(str(x)))
df2['embedding'] = df2['Plant Name'].apply(lambda x: model.encode(str(x)))

# 4) Find similar pairs based on a similarity threshold
similar_pairs = []
threshold = 0.80  # Adjust this as needed

for i, row1 in df1.iterrows():
    emb1 = row1['embedding'].reshape(1, -1)

    for j, row2 in df2.iterrows():
        emb2 = row2['embedding'].reshape(1, -1)

        # Compute cosine similarity
        sim = cosine_similarity(emb1, emb2)[0][0]

        # If above threshold, record the pair
        if sim >= threshold:
            similar_pairs.append({
                'df1_name': row1['botanical name'],
                'df2_name': row2['Plant Name'],
                'similarity': sim
            })

# 5) Convert the list of similar pairs to a DataFrame
result_df = pd.DataFrame(similar_pairs)

# 6) Print or inspect the result
print(result_df)


In [ ]:
result_df.shape

(22, 3)

In [ ]:
df3 = pd.read_csv("tree_species.csv")
df3 = df3["Botanical Name"]
df3.head()

,Botanical Name
0,Pterocarpus Marsupium
1,Haldina Cordifolia
2,Terminalia Chebula
3,Terminalia Bellirica
4,Terminalia Arjuna


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# ----------------------------------------------------
# 1) Read your three datasets
#    Adjust file paths and column names to match your files
# ----------------------------------------------------
df1 = pd.read_excel("name_table.xls")               # has column 'botanical name'
df2 = pd.read_csv("bsi_plant_data_formatted.csv")   # has column 'Plant Name'
df3 = pd.read_csv("tree_species.csv")               # has column 'Botanical Name'

# ----------------------------------------------------
# 2) Load the SentenceTransformer model
#    You can choose any model from huggingface.co
# ----------------------------------------------------
model = SentenceTransformer('all-MiniLM-L6-v2')

# ----------------------------------------------------
# 3) Create embeddings for df1, df2, df3
#    Convert each plant name to a vector
# ----------------------------------------------------
df1['embedding'] = df1['botanical name'].apply(lambda x: model.encode(str(x)))
df2['embedding'] = df2['Plant Name'].apply(lambda x: model.encode(str(x)))
df3['embedding'] = df3['Botanical Name'].apply(lambda x: model.encode(str(x)))

# ----------------------------------------------------
# 4) Merge df1 and df2 into a single union,
#    with no duplicates above a similarity threshold
#    "Intersection only once and that too of the bsi one (df2)."
# ----------------------------------------------------
threshold = 0.80  # adjust as needed

# We'll store the final rows in a list of dicts
union_rows = []
matched_df2_indices = set()  # which rows in df2 we've already matched

# For each entry in df1, find the best match in df2
for i, row1 in df1.iterrows():
    emb1 = row1['embedding'].reshape(1, -1)

    # Compute similarities with all rows in df2
    sims = []
    for j, row2 in df2.iterrows():
        emb2 = row2['embedding'].reshape(1, -1)
        sim = cosine_similarity(emb1, emb2)[0][0]
        sims.append((sim, j, row2))

    # Find the row in df2 with the highest similarity to row1
    best_sim, best_j, best_row2 = max(sims, key=lambda x: x[0])

    if best_sim >= threshold:
        # It's considered the "same" plant, so keep the df2 version
        # Mark that df2 row as matched
        matched_df2_indices.add(best_j)

        # We'll add the df2 row to the union if not already added
        union_rows.append({
            'plant_name': best_row2['Plant Name'],
            'embedding': best_row2['embedding'],
            'source': 'df2 (intersection)'
        })
    else:
        # No good match in df2, so keep the df1 entry
        union_rows.append({
            'plant_name': row1['botanical name'],
            'embedding': row1['embedding'],
            'source': 'df1'
        })

# Also add any df2 rows that were never matched to df1
for j, row2 in df2.iterrows():
    if j not in matched_df2_indices:
        union_rows.append({
            'plant_name': row2['Plant Name'],
            'embedding': row2['embedding'],
            'source': 'df2 (unmatched)'
        })

# Convert union_rows to a DataFrame
df12_union = pd.DataFrame(union_rows)

# ----------------------------------------------------
# 5) Check if df3 entries are found in df12_union
#    We'll see which names in df3 are "new"
#    (i.e., no match in df12_union above threshold)
# ----------------------------------------------------
not_found_in_union = []

for i, row3 in df3.iterrows():
    emb3 = row3['embedding'].reshape(1, -1)

    # Compare to every row in df12_union
    sims_union = []
    for k, union_row in df12_union.iterrows():
        emb_union = union_row['embedding'].reshape(1, -1)
        sim = cosine_similarity(emb3, emb_union)[0][0]
        sims_union.append(sim)

    best_sim_union = max(sims_union) if sims_union else 0.0

    # If best similarity is below threshold, it's not found
    if best_sim_union < threshold:
        not_found_in_union.append(row3['Botanical Name'])

# ----------------------------------------------------
# 6) Print or analyze results
# ----------------------------------------------------
print("Merged df1 + df2 (df12_union):")
print(df12_union[['plant_name', 'source']])

print("\nNames in df3 that are NOT in df12_union (above similarity threshold):")
for name in not_found_in_union:
    print("  -", name)


Merged df1 + df2 (df12_union):
                                      plant_name              source
0                      Aegle marmelos (L.) Corr.  df2 (intersection)
1                    Alpinia galanga (L.) Willd.  df2 (intersection)
2                          Amaranthus viridis L.  df2 (intersection)
3                  Artocarpus heterophyllus Lam.  df2 (intersection)
4                Azadirachta indica (L.) A.Juss.  df2 (intersection)
...                                          ...                 ...
1933  Ziziphus nummularia (Burm.f.) Wight & Arn.     df2 (unmatched)
1934                Ziziphus oenoplia (L.) Mill.     df2 (unmatched)
1935                        Ziziphus rugosa Lam.     df2 (unmatched)
1936                        Zornia gibbosa Span.     df2 (unmatched)
1937                      Zygophyllum simplex L.     df2 (unmatched)

[1938 rows x 2 columns]

Names in df3 that are NOT in df12_union (above similarity threshold):
  - Terminalia Chebula
  - Terminalia Arjuna


In [ ]:
df12_union.shape

(1938, 3)

In [ ]:
import os
import shutil

# Path to the "images" folder in Google Colab
base_path = "/content/images"

# List of plant names to be deleted
folders_to_delete = [
    "Aegle marmelos", "Alpinia Galanga", "Amaranthus Viridis", "Artocarpus Heterophyllus",
    "Azadirachta Indica", "Brassica Juncea", "Butea Monosperma", "Crotalaria juncea -(Indian",
    "Ficus Religiosa", "Hibiscus Rosa", "Lycium barbarum", "Mangifera Indica", "Moringa Oleifera",
    "Murraya Koenigii", "Psidium Guajava", "Punica Granatum", "Santalum Album", "Syringa Vulgaris",
    "Syzygium Cumini", "Syzygium Jambos", "Thespesia populnea", "Trigonella Foenum"
]

# Loop through each folder and delete if it exists
for folder in folders_to_delete:
    folder_path = os.path.join(base_path, folder)
    if os.path.exists(folder_path) and os.path.isdir(folder_path):
        shutil.rmtree(folder_path)
        print(f"Deleted: {folder}")
    else:
        print(f"Not found: {folder}")
